In [3]:
import pandas as pd
from collections import Counter, defaultdict
from us import states
from utils import get_names2abbrs_dict

In [21]:
# pres_df = pd.read_csv(open('poll_data/president_polls.csv'))
pres_df = pd.read_csv('president_polls_latest.csv')

/usr/local/Caskroom/miniconda/base/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,22,25,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
pres_df

,question_id,poll_id,cycle,state,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,...,created_at,notes,url,stage,race_id,answer,candidate_id,candidate_name,candidate_party,pct
0,136283,72621,2020,Iowa,383,Public Policy Polling,NaN,NaN,Public Policy Polling,263.0,...,11/2/20 09:02,NaN,https://www.publicpolicypolling.com/wp-content...,general,6223,Biden,13256,Joseph R. Biden Jr.,DEM,49.0
1,136283,72621,2020,Iowa,383,Public Policy Polling,NaN,NaN,Public Policy Polling,263.0,...,11/2/20 09:02,NaN,https://www.publicpolicypolling.com/wp-content...,general,6223,Trump,13254,Donald Trump,REP,48.0
2,136322,72647,2020,Pennsylvania,461,Susquehanna Polling & Research Inc.,NaN,NaN,Susquehanna Polling & Research Inc.,326.0,...,11/2/20 12:49,NaN,https://www.realclearpolitics.com/docs/2020/Su...,general,6249,Biden,13256,Joseph R. Biden Jr.,DEM,48.4
3,136322,72647,2020,Pennsylvania,461,Susquehanna Polling & Research Inc.,NaN,NaN,Susquehanna Polling & Research Inc.,326.0,...,11/2/20 12:49,NaN,https://www.realclearpolitics.com/docs/2020/Su...,general,6249,Trump,13254,Donald Trump,REP,49.2
4,136322,72647,2020,Pennsylvania,461,Susquehanna Polling & Research Inc.,NaN,NaN,Susquehanna Polling & Research Inc.,326.0,...,11/2/20 12:49,NaN,https://www.realclearpolitics.com/docs/2020/Su...,general,6249,Jorgensen,14611,Jo Jorgensen,LIB,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16433,92081,57026,2020,Ohio,383,Public Policy Polling,1363,DEM (partisan),Public Policy Polling,263.0,...,12/6/18 14:45,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Trump,13254,Donald Trump,REP,46.0
16434,92082,57026,2020,Ohio,383,Public Policy Polling,1363,DEM (partisan),Public Policy Polling,263.0,...,12/6/18 14:46,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Warren,13258,Elizabeth Warren,DEM,43.0
16435,92082,57026,2020,Ohio,383,Public Policy Polling,1363,DEM (partisan),Public Policy Polling,263.0,...,12/6/18 14:46,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Trump,13254,Donald Trump,REP,49.0
16436,92078,57025,2020,NaN,399,Rasmussen Reports/Pulse Opinion Research,NaN,NaN,Rasmussen Reports/Pulse Opinion Research,277.0,...,12/6/18 14:40,NaN,http://www.rasmussenreports.com/public_content...,general,6210,Obama,13253,Michelle Obama,DEM,50.0


In [23]:
def clean_pres_polls(in_df):
    df = in_df.copy()
    polls = list()
    names2abbrs = get_names2abbrs_dict()
    column_list = ['question_id', 'poll_id', 'state', 'fte_grade', 'sample_size', 
               'population', 'population_full', 'methodology', 'start_date', 
               'end_date', 'internal', 'partisan', 'tracking', 'created_at', 
               'url', 'answer', 'candidate_name', 'candidate_party', 'pct']
    for p_id in Counter(df['poll_id']):
        for q_id in Counter(df[df['poll_id'] == p_id]['question_id']):
            mask = (df['poll_id'] == p_id) & (df['question_id'] == q_id) & (df['cycle'] == 2020) & (df['stage'] == 'general')
            rows = df[mask]
            rows = df[column_list]
            if set(rows['candidate_name']) == set(['Joseph R. Biden Jr.', 'Donald Trump']) and len(rows) == 2:
                state = list(set(rows['state']))
                assert len(state) == 1
                state_str = str(state[0])
                if state_str != 'nan':
                    state_str = names2abbrs[state_str]
                else:
                    state_str = 'NAT'
                rows['state_po'] = state_str
                polls.extend(rows)
    df = pd.concat(polls)

In [24]:
def get_pres_poll_d_prob(in_df):
    df = in_df.copy()
    results = list()
    names2abbrs = get_names2abbrs_dict()
    for p_id in Counter(df['poll_id']):
        for q_id in Counter(df[df['poll_id'] == p_id]['question_id']):
            mask = (df['poll_id'] == p_id) & (df['question_id'] == q_id) & (df['cycle'] == 2020) & (df['stage'] == 'general')
            rows = df[mask]
            if set(rows['candidate_name']) == set(['Joseph R. Biden Jr.', 'Donald Trump']) and len(rows) == 2:
                state = list(set(rows['state']))
                assert len(state) == 1
                state_str = str(state[0])
                if state_str != 'nan':
                    state_po = names2abbrs[state_str]
                else:
                    state_po = 'NAT'
                assert len(set(rows['end_date'])) == 1
                date = rows['end_date'].iloc[0]
                month, day, year = date.split('/')
                month = month.zfill(2)
                day = day.zfill(2)
                year = f'20{year}'
                date = '-'.join([year, month, day])  # get yyyy-mm-dd sortable format
                sample_size = rows['sample_size'].iloc[0].item()
                population = rows['population'].iloc[0]
                
                d_row = rows[rows['candidate_party'] == 'DEM']
                r_row = rows[rows['candidate_party'] == 'REP']
                assert len(d_row) == 1 and len(r_row) == 1
                d_pct = d_row['pct'].item()
                r_pct = r_row['pct'].item()
                d_prob = d_pct / (d_pct + r_pct)
                result = [date, p_id, q_id, state_str, state_po, sample_size, population, d_prob]
                results.append(result)
    return pd.DataFrame(results, columns=['date', 'poll_id', 'question_id', 'state', 'state_po', 'sample_size', 'population', 'd_prob']).sort_values(by='date').reset_index(drop=True)

In [25]:
d_probs = get_pres_poll_d_prob(pres_df)

In [26]:
d_probs = d_probs.loc[list(d_probs['sample_size'].dropna().index)].reset_index(drop=True).sort_values(by='date')

In [27]:
'question_id', 'poll_id', 'state', 'fte_grade', 'sample_size', 
'population', 'population_full', 'methodology', 'start_date', 
'end_date', 'internal', 'partisan', 'tracking', 'created_at', 
'url', 'race_id', 'answer', 'candidate_name', 'candidate_party', 
'pct'

'pct'

In [29]:
d_probs.to_csv('./poll_data/all_pres_polls_d_probs.csv', index=False)